In [1]:
import pandas as pd 
import numpy as np
import ast
import re
from bs4 import BeautifulSoup
import PyPDF2 # type: ignore
from sentence_transformers import SentenceTransformer, util # type: ignore

In [2]:
model = SentenceTransformer('all-mpnet-base-v2')

#### Tags Text Cleaning

In [3]:
def clean_categories(categories):
    tags = [tag.strip() for tag in ast.literal_eval(categories)]
    return tags

In [4]:
df = pd.read_csv(r'arxiv-paper-abstracts\arxiv_data_210930-054931.csv')
categories = pd.DataFrame(list(df['terms'].unique()),columns=['categories'])
categories['clean_categories'] = categories['categories'].apply(clean_categories)
tags = set()
for index, row in categories.iterrows():
    categories = row['clean_categories']
    for category in categories:
        tags.add(category)
tags_df = pd.DataFrame(list(tags), columns=['tags'])
tags = set()
for index, row in tags_df.iterrows():
    categories_str = row['tags']
    categories_list = [category.strip() for category in categories_str.replace(',', ';').split(';')]
    for category in categories_list:
        tags.add(category)
tags_df = pd.DataFrame(list(tags), columns=['tags'])
tags = set()
for index, row in tags_df.iterrows():
    categories_str = row['tags']
    categories_list = [category.strip() for category in categories_str.replace(',', ';').split(';')]
    for category in categories_list:
        if '(' in category and ')' in category:
            subcategories = [subcat.strip() for subcat in category.split() if subcat[0].isdigit()]
            for subcategory in subcategories:
                tags.add(subcategory)
        else:
            tags.add(category)
tags.discard('')


In [5]:
tags_list = []
for tag in tags:
    tag = tag.replace('(Primary)', '')
    tag = tag.replace('Primary', '')
    tag = tag.replace('(Secondary)', '')
    tag = tag.replace('(secondary)', '')
    tag = tag.replace('secondary', '')
    tag = tag.replace('Secondary', '')
    tag = tag.replace('and ', '')
    tags_list.append(tag.strip())
print(tags_list)

['93E03', '60L10', '62P10', '14M07', 'C.4', '46M20', 'q-fin.PM', 'H.2.7', 'I.7.1', '68U35', '92C55', 'cond-mat.mes-hall', '68: computer science', 'H.2.8', '68M25', 'cs.NA', '15A83', 'C.2.6', '60J05', '93E20', 'I.2.2', 'physics.optics', '86-04', '60J27', 'math-ph', '90', '90-01', '65M20', '68T09', '35K15', '15A29', '51K05', 'math.DG', '2010', 'cs.SY', '65R32', '91G80', '60G35', 'I.2.9', '35J08', '86A10', '65N21', 'physics.geo-ph', '14J60', 'cond-mat.str-el', '35Q84', '62G10', '92B20', 'D.2.11', '05C80', 'cond-mat.dis-nn', '60B20', 'K.3.2', '65M22', '62F07', '74L05', '90C90', '49N10', '62P12', 'F.m', '92B25 92F99', '49-06', '62F15', '62R40', 'astro-ph', 'I.2.0.b', '00-02', 'H.4.m', 'math.NA', 'math.MG', '90C35', '65N12', 'C.1.3', 'I.4.7', 'math.PR', '74Q15', '68T05', 'q-bio.QM', '41A52', '35A18', '62G32', '65F50', 'C.4.4', 'I.2.4', '49M41', '68T07', '68T01', 'G.3.11', '60D05', '35Q79', 'physics.acc-ph', '65R30', '78', 'I.4.0', '30E05', '53Z50', 'cond-mat.other', '91A12', 'cs.CR', 'cs.OS'

In [6]:
# create empty lists for each classification
msc = set()
ccs = set()
arxiv = set()
misc = set()

for tag in tags_list:
    if re.match(r'\d{2}[A-Z]\d{2}|\d{2}[A-Z]-xx|\d{2}\w{2,}', tag):
        msc.add(tag)
    elif re.match(r'\d{2}-\d{2}|\d{2}-xx|\d{2}-XX', tag):
        msc.add(tag)
    elif re.match(r'\d{2}', tag):
        msc.add(tag)
    elif re.match(r'[A-Z]\.\d\.\d', tag):
        ccs.add(tag)
    elif re.match(r'[A-Z]\.\d', tag):
        ccs.add(tag)
    elif re.match(r'[A-Z]\d\.\d', tag):
        ccs.add(tag)
    elif re.match(r'\w+\.\w+', tag):
        arxiv.add(tag)
    elif re.match(r'\w+-\w+\.\w+-\w+', tag):
        arxiv.add(tag)
    elif re.match(r'\w+-\w+\.\w+', tag):
        arxiv.add(tag)
    elif re.match(r'\w+-\w+', tag):
        arxiv.add(tag)
    else:
        misc.add(tag)


In [7]:
print(f'MSC: {len(msc)}\nCCS: {len(ccs)}\nArXiv: {len(arxiv)}\nMisc: {len(misc)}')

MSC: 453
CCS: 162
ArXiv: 149
Misc: 5


**Contents of misc:**
- Artificial intelligence': corresponds to 68Txx
- Computer science applications- 97R60 Computer graphics': corresponds to 97R60
- Computing methodologies for image processing': corresponds to 68U10
- This paper tells us how human can be identified by their Gait cycle\n  using any simple camera': dropped
- org: dropped

In [8]:
msc.add('97R60')
del misc

In [58]:
msc_cleaned = pd.DataFrame(list(msc), columns=['msc'])
ccs_cleaned = pd.DataFrame(list(ccs), columns=['ccs'])
arxiv_cleaned = pd.DataFrame(list(arxiv), columns=['arxiv'])

#### Retrieve Descriptions for Each Tag

In [59]:
msc_html = r'C:\Users\User\Desktop\ml_paper_recommender_system\paper-recommender\Classification Codes Data\MSC Classification Codes.html'

# Parse MSC Classification Codes html file
with open(msc_html) as fp:
    msc_soup = BeautifulSoup(fp, 'html.parser')

# Extract MSC Classification Codes
msc_tree = [element.text for element in msc_soup.find_all('li')]

# Define Dictionary for MSC Classification Codes where the key is the main class and the value is a list of sub-classes
msc_classes = {}

# Populate the dictionary
for cls in msc_tree:
    if re.match(r'^\d\d-xx:', cls):
        msc_classes[cls.split('\n')[0]] = []

for element in msc_tree:
    for cls in msc_classes.keys():
        if cls[0:2] == element[0:2]:
            msc_classes[cls].append(element)

for cls in msc_classes.keys():
    msc_classes[cls].pop(0)

# Create a dataframe from the dictionary
msc_df = pd.DataFrame(msc_classes.items(),columns=['class','sub_classes'])

In [60]:
for i, subclass in enumerate(msc_df['sub_classes']):
    for idx, sub in enumerate(subclass):
        if re.match(r'\d+\d+[A-Z]xx', sub):
            msc_df['sub_classes'][i][idx] = sub.split('\n')[0]

In [61]:
for i, subclass in enumerate(msc_df['sub_classes']):
    curr_cls = msc_df['class'][i]
    for idx, sub in enumerate(subclass):
        msc_df['sub_classes'][i][idx] = curr_cls + ' ' + msc_df['sub_classes'][i][idx]

In [62]:
for idx, tag in enumerate(msc_cleaned['msc']):
    if ':' in tag:
        msc_cleaned['msc'][idx] = tag.split(':')[0]

In [63]:
msc_cleaned['description'] = None

In [64]:
for i, tag in enumerate(msc_cleaned['msc']):
    for j, subclass in enumerate(msc_df['sub_classes']):
        for k, cls in enumerate(subclass):
            if tag in cls:
                msc_cleaned['description'][i] = cls

In [65]:
missing_msc = []
for idx, description in enumerate(msc_cleaned['description']):
    if description is None:
        missing_msc.append(msc_cleaned['msc'][idx])
print(f'Status:\n{len(missing_msc)} out of {len(msc_cleaned)} tags are missing a description')

Status:
72 out of 454 tags are missing a description


In [66]:
# Extract Description from MSC Classification Codes 2020
reader = PyPDF2.PdfReader(r'C:\Users\User\Desktop\ml_paper_recommender_system\paper-recommender\Classification Codes Data\msc2020.pdf')
missing_corpus = []
for idx, page in enumerate(reader.pages):
    text = page.extract_text()
    for idx, code in enumerate(missing_msc):
        if code in text:
            missing_corpus.append(text)
            # print(f'Page {idx+1}: {code}')

In [67]:
for idx, text in enumerate(missing_corpus):
    missing_corpus[idx] = text.split('\n')

In [68]:
missing_descriptions = set()
for code in missing_msc:
    for idx, text in enumerate(missing_corpus):
        for line in text:
            if code in line:
                desc = f'{code}: {line}'
                missing_descriptions.add(desc)

In [75]:
# Inspected manually because data quality right?
missing_descriptions = [
 '60B20: 60B20 Random matrices (probabilistic aspects)',
 '62H22: 62H22 Probabilistic graphical models',
 '97P30: 97P30 Systems, databases (educational aspects)',
 '05C81: 05C81 Random walks on graphs',
 '62B86: 62B86 Statistical aspects of fuzziness, sufficiency, and information',
 '68T42: 68T42 Agent technology and artificial intelligence',
 '97N80: 97N80 Mathematical software, computer programs (educational aspects)',
 '60L10: 60L10 Signatures and data streams',
 '35Q68: 35Q68 PDEs in connection with computer science',
 '68T07: 68T07 Artificial neural networks and deep learning',
 '60L20: 60L20 Rough paths',
 '62-08: 62-08 Computational methods for problems pertaining to statistics',
 '35Q79: 35Q79 PDEs in connection with classical thermodynamics and heat transfer',
 '15B48: 15B48 Positive matrices and their generalizations; cones of matrices',
 '68W50: 68W50 Evolutionary algorithms, genetic algorithms (computational aspects)',
 '62-XX: 62-XX Statistics',
 '65Zxx: 65Zxx Applications to the sciences',
 '62G86: 62G86 Nonparametric inference and fuzziness',
 '62R40: 62R40 Topological data analysis',
 '05C21: 05C21 Flows in graphs',
 "35J08: 35J08 Green's functions for elliptic equations",
 '91-10: 91-10 Mathematical modeling or simulation for problems pertaining to game theory, economics, and Finance',
 '90-05: 90-05 Experimental work for problems pertaining to operations research and mathematical programming',
 '68W27: 68W27 Online algorithms; streaming algorithms',
 '14T10: 14T10 Foundations of tropical geometry and relations with algebra',
 '62D20: 62D20 Causal inference from observational studies',
 '49Q22: 49Q22 Optimal transportation',
 '35R02: 35R02 PDEs on graphs and networks (ramified or polygonal spaces)',
 '57-08: 57-08 Computational methods for problems pertaining to manifolds and cell complexes',
 '65F55: 65F55 Numerical methods for low-rank matrix approximation; matrix compression',
 '90C17: 90C17 Robustness in mathematical programming',
 '53Z50: 53Z50 Applications of differential geometry to data and computer science',
 '35J47: 35J47 Second-order elliptic systems',
 '62R07: 62R07 Statistical aspects of big data and data science',
 '15A83: 15A83 Matrix completion problems',
 '91G80: 91G80 Financial applications of other theories',
 '54H30: 54H30 Applications of general topology to computer science (e.g., digital topology, image processing)',
 '35K08: 35K08 Heat kernel',
 '90-10: 90-10 Mathematical modeling or simulation for problems pertaining to operations research and mathematical programming',
 '78M32: 78M32 Neural and heuristic methods applied to problems in optics and electromagnetic theory',
 '62A09: 62A09 Graphical methods in statistics',
 '35Q74: 35Q74 PDEs in connection with mechanics of deformable solids',
 '55N31: 55N31 Persistent homology and applications, topological data analysis',
 '68-XX: 68-XX Computer science',
 '65D19: 65D19 Computational issues in computer and robotic vision',
 '68M25: 68M25 Computer security',
 '62-08: 62-08 Computational methods for problems pertaining to statistics',
 '92-XX: 92-XX Biology and other natural sciences',
 '35-XX: 35-XX Partial differential equations',
 '68Q87: 68Q87 Probability in computer science (algorithm analysis, random structures, phase transitions, etc.)',
 '60B20: 15B52 Random matrices (algebraic aspects) fFor probabilistic aspects, see 60B20 g',
 '91G70: 91G70 Statistical methods; risk measures [See also 62P05, 62P20]',
 '62R01: 62R01 Algebraic statistics',
 '49Q22: 35Q49 Transport equations fFor calculus of variations and optimal control, see 49Q22; for Fluid mechanics, see',
 '60B20: 60B20 Random matrices (probabilistic aspects)',
 '49M41: 49M41 PDE constrained optimization (numerical aspects)',
 '57Z25: 57Z25 Relations of manifolds and cell complexes with computer and data science',
 '49-11: 49-11 Research data for problems pertaining to calculus of variations and optimal control',
 '35Q84: 35Q84 Fokker-Planck equations',
 '42-08: 42-08 Computational methods for problems pertaining to harmonic analysis on Euclidean spaces',
 '68T09: 68T09 Computational aspects of data analysis and big data',
 '65M22: 65M22 Numerical solution of discretized equations for initial value and initial-boundary value problems involving ']

for idx, desc in enumerate(missing_descriptions):
    missing_descriptions[idx] = desc.split(':')


for idx, description in enumerate(missing_descriptions):
    m, d = missing_descriptions[idx]
    for idx2, code in enumerate(msc_cleaned['msc']):
        if msc_cleaned['msc'][idx2] == m:
            msc_cleaned['description'][idx2] = d.strip()
    

In [ ]:
# # Add row to dataframe
# msc_cleaned.loc[-1] = [m, d.strip()]
# msc_cleaned.index = msc_cleaned.index + 1
# msc_cleaned = msc_cleaned.sort_index()

In [82]:
missing_descriptions.sort()
missing_descriptions[50:-1]  

[['68W27', ' 68W27 Online algorithms; streaming algorithms'],
 ['68W50',
  ' 68W50 Evolutionary algorithms, genetic algorithms (computational aspects)'],
 ['78M32',
  ' 78M32 Neural and heuristic methods applied to problems in optics and electromagnetic theory'],
 ['90-05',
  ' 90-05 Experimental work for problems pertaining to operations research and mathematical programming'],
 ['90-10',
  ' 90-10 Mathematical modeling or simulation for problems pertaining to operations research and mathematical programming'],
 ['90C17', ' 90C17 Robustness in mathematical programming'],
 ['91-10',
  ' 91-10 Mathematical modeling or simulation for problems pertaining to game theory, economics, and Finance'],
 ['91G70',
  ' 91G70 Statistical methods; risk measures [See also 62P05, 62P20]'],
 ['91G80', ' 91G80 Financial applications of other theories'],
 ['92-XX', ' 92-XX Biology and other natural sciences'],
 ['97N80',
  ' 97N80 Mathematical software, computer programs (educational aspects)']]

In [76]:
msc_cleaned[msc_cleaned['description'].isnull()] # TODO fetch

,msc,description
18,2010,None
36,92B25 92F99,None
56,97R60,None
127,68T04,None
139,62h30,None
145,35CXX,None
155,6006,None
228,10010147.10010257.10010258.10010259.10010263,None
233,62-07l,None
288,6804,None
